In [1]:
getwd()
setwd(paste0(getwd(), '/..'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/NA_fix"

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

In [43]:
source('helpers.R')

kaggle.house.loadLibraries()
data = kaggle.house.loadData()

df.combined = data$combined %>% filter(!is.na(LotFrontage))
df.combined %>% count

n
2433


In [41]:
df.combined %>% 
select_if(is.numeric) %>%
gather(attr.name, attr.val, -LotFrontage) %>%
mutate(
    attr.val.log = ifelse(attr.val > 0, log(attr.val), 0),
    LotFrontage.log = log(LotFrontage)
) -> df.combined.long

df.combined.long %>% head
df.combined.long %>% count

LotFrontage,attr.name,attr.val,attr.val.log,LotFrontage.log
65,LotArea,8450,9.041922,4.174387
80,LotArea,9600,9.169518,4.382027
68,LotArea,11250,9.328123,4.219508
60,LotArea,9550,9.164296,4.094345
84,LotArea,14260,9.565214,4.430817
85,LotArea,14115,9.554993,4.442651


n
82722


In [37]:
df.combined.long %>% 
group_by(attr.name) %>% 
nest %>% 
mutate(
    mod = map(data, ~lm(LotFrontage ~ attr.val, .)), 
    glance = map(mod, broom::glance),
    rsq = glance %>% map_dbl("r.squared"),
    
    mod2 = map(data, ~lm(LotFrontage ~ attr.val.log, .)), 
    glance2 = map(mod2, broom::glance),
    rsq2 = glance2 %>% map_dbl("r.squared"),
    
    mod3 = map(data, ~lm(LotFrontage.log ~ attr.val.log, .)), 
    glance3 = map(mod3, broom::glance),
    rsq3 = glance3 %>% map_dbl("r.squared")
) %>%
arrange(desc(rsq)) %>%
select(attr.name, rsq, rsq2, rsq3) %>%
print

# A tibble: 34 x 4
      attr.name        rsq        rsq2        rsq3
          <chr>      <dbl>       <dbl>       <dbl>
 1      LotArea 0.23999767 0.467189030 0.587193938
 2    X1stFlrSF 0.20999074 0.210587817 0.218847392
 3    GrLivArea 0.14627714 0.125415343 0.108557166
 4   GarageArea 0.12944610 0.033312216 0.033526172
 5  TotalBsmtSF 0.12589894 0.008013407 0.007024319
 6 TotRmsAbvGrd 0.12215963 0.123642972 0.126792650
 7   GarageCars 0.09646429 0.090350458 0.077319103
 8   Fireplaces 0.06862818 0.030197358 0.022495100
 9 BedroomAbvGr 0.05517420 0.060886185 0.074960584
10   MasVnrArea 0.04887582 0.038520767 0.023899711
# ... with 24 more rows


In [45]:
df.combined %>% 
select_if(is.numeric) %>%
gather(attr.name, attr.val, -LotFrontage, -LotArea) %>%
mutate(
    attr.val.log = ifelse(attr.val > 0, log(attr.val), 0),
    LotFrontage.log = log(LotFrontage),
    LotArea.log = log(LotArea)
) %>%
select(-LotFrontage, -LotArea) -> df.combined.long2

df.combined.long2 %>% head
df.combined.long2 %>% count

attr.name,attr.val,attr.val.log,LotFrontage.log,LotArea.log
OverallQual,7,1.945910,4.174387,9.041922
OverallQual,6,1.791759,4.382027,9.169518
OverallQual,7,1.945910,4.219508,9.328123
OverallQual,7,1.945910,4.094345,9.164296
OverallQual,8,2.079442,4.430817,9.565214
OverallQual,5,1.609438,4.442651,9.554993


n
80289


In [46]:
df.combined.long2 %>% 
group_by(attr.name) %>% 
nest %>% 
mutate(
    mod1 = map(data, ~lm(LotFrontage.log ~ LotArea.log + attr.val, .)), 
    glance1 = map(mod1, broom::glance),
    rsq1 = glance1 %>% map_dbl("r.squared"),
    
    mod2 = map(data, ~lm(LotFrontage.log ~ LotArea.log + attr.val.log, .)), 
    glance2 = map(mod2, broom::glance),
    rsq2 = glance2 %>% map_dbl("r.squared")
) %>%
arrange(desc(rsq1)) %>%
select(attr.name, rsq1, rsq2) %>%
print

# A tibble: 33 x 3
      attr.name      rsq1      rsq2
          <chr>     <dbl>     <dbl>
 1    X1stFlrSF 0.5952974 0.5986197
 2   GarageArea 0.5932784 0.5883477
 3  TotalBsmtSF 0.5926353 0.5885418
 4   MasVnrArea 0.5922331 0.5943946
 5 TotRmsAbvGrd 0.5918779 0.5929072
 6   GarageCars 0.5915473 0.5908690
 7 BedroomAbvGr 0.5912319 0.5936016
 8    BsmtUnfSF 0.5912117 0.5895906
 9    GrLivArea 0.5891953 0.5892236
10  OverallQual 0.5888413 0.5885256
# ... with 23 more rows
